## Loading Modules

In [1]:
%matplotlib inline
import import_ipynb
import warnings
import datacube
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from datacube.utils import geometry
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from deafrica_tools.bandindices import calculate_indices
# from deafrica_tools.coastal import tidal_tag
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import rgb, display_map
from datacube.utils.geometry import Geometry
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.areaofinterest import define_area

In [2]:
create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 12,Total memory: 12.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36745,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 12.15 GiB
Comm: tcp://127.0.0.1:39401,Total threads: 12
Dashboard: /proxy/35533/status,Memory: 12.15 GiB
Nanny: tcp://127.0.0.1:33275,


In [3]:
dc = datacube.Datacube(app="Mangrove")

In [4]:
# Method 1: Specify the latitude, longitude, and buffer
# aoi = define_area(lat=22.4, lon=69.5, buffer=0.15)

# # Method 2: Use a polygon as a GeoJSON or Esri Shapefile.
# #aoi = define_area(shapefile_path='aoi.shp')

# #Create a geopolygon and geodataframe of the area of interest
# geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
# geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)

# Get the latitude and longitude range of the geopolygon
# lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
# lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

# product_name = "s2_l2a"

# time_range = ("2017", "2021")

# tide_range = (0.25, 0.75) # only keep mid tides
 
# lat_range = (22.233429508811017, 22.408743282564075)
# lon_range = (69.29771974035943, 69.56833033401843)
lat_range = (15.65418332, 15.95828652)
lon_range = (80.69694696, 81.09203692)
product_name = "s2a_sen2cor_granule"
time_range = ('2019-01-15', '2023-02-15')
tide_range = (0.25, 0.75)

In [5]:
display_map(lon_range, lat_range)

/home/adithya/anaconda3/envs/odc_env/lib/python3.8/site-packages/deafrica_tools/plotting.py:284: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  all_longitude, all_latitude = transform(Proj(crs), Proj("EPSG:4326"), all_x, all_y)


In [8]:
# create a query dict for the datacube
query = {
    "time": time_range,
    'x': lon_range,
    'y': lat_range,
    "group_by": "solar_day",
    "resolution": (-30, 30),
    "output_crs":"EPSG:6933"
}

# load data
# ds = load_ard(
#     dc=dc,
#     products=[product_name],
#     measurements=["red", "nir", "green", "blue"], #use nir-narrow for NDVI
#     mask_filters=[("opening", 5), ("dilation", 5)],
#     dask_chunks={"time": 1, "x": 1000, "y": 1000},
#     **query,
# )


ds = dc.load(product = product_name, measurements=["red", "nir", "green", "blue"], **query)

# ds = load_ard(
#     dc=dc,
#     products=[product_name],
#     mask_filters=[("opening", 5), ("dilation", 10)],
#     **query,
# )
# ds = dc.load(product="s2a_sen2cor_granule",
#                  measurements=["B04_10m","B03_10m","B02_10m", "B08_10m", "B11_20m"],
#              x=lon_range,
#              y=lat_range,
#              time=time_range,
#              output_crs='EPSG:6933',
#              resolution=(-30, 30))
print(ds)

AttributeError: 'NoneType' object has no attribute 'begin'

In [ ]:
#Rasterise the area of interest polygon
# aoi_raster = xr_rasterize(gdf=geopolygon_gdf,da=ds, transform=ds.geobox.transform,
#                                    crs=ds.crs)
# #Mask the dataset to the rasterised area of interest
# ds = ds.where(aoi_raster==1)

In [ ]:
gmw = dc.load(product='gmw_latest',
              like=ds.geobox,
              time='2015')
gmw

In [ ]:
gmw.mangrove.plot(figsize=(6,6));

In [ ]:
#calculate NDVI, rename nir2 to trick calculate_indices
# ds_filtered = ds_filtered.rename({'nir_2':'nir'})
ds_filtered = calculate_indices(ds, index='NDVI', satellite_mission='s2')

In [ ]:
# generate median annual summaries of NDVI
ds_summaries = ds_filtered.NDVI.groupby("time.year").median().compute()

# Plot the output summary images
ds_summaries.plot(col="year", cmap="YlGn", col_wrap=len(ds_summaries.year.values), vmin=0, vmax=1.0);

In [ ]:
# Mask dataset to set pixels outside the GMW layer to `NaN`
ds_summaries_masked = ds_summaries.where(gmw.mangrove.squeeze())

In [ ]:
all_mangroves = xr.where(ds_summaries_masked > 0.4, 1, np.nan)

regular_mangroves = all_mangroves.where(ds_summaries_masked <= 0.7)
closed_mangroves = all_mangroves.where(ds_summaries_masked > 0.7)

In [ ]:
mangroves = xr.concat(
    [regular_mangroves, closed_mangroves, all_mangroves],
    dim=pd.Index(["regular", "closed", "total"], name="mangrove_type"),
)

# regular_color = "gold"
# closed_color = "green"

# # Create a FacetGrid, so there is a subplot for each year
# grid = xr.plot.FacetGrid(
#     mangroves,
#     col="year",
#     col_wrap=len(ds_summaries.year.values),
#     size=6,
#     aspect=mangroves.x.size / mangroves.y.size,
# )

# # Define the sub-plot of mangrove types with a legend on a background of grey-scale NDVI
# def plot_mangrove(data, ax, **kwargs):
#     ds_summaries.sel(year=data.year).plot.imshow(
#         ax=ax, cmap="Greys", vmin=-1, vmax=1, add_colorbar=False, add_labels=False
#     )
#     data.sel(mangrove_type="regular").plot.imshow(
#         ax=ax,
#         cmap=ListedColormap([regular_color]),
#         add_colorbar=False,
#         add_labels=False,
#     )
#     data.sel(mangrove_type="closed").plot.imshow(
#         ax=ax, cmap=ListedColormap([closed_color]), add_colorbar=False, add_labels=False
#     )

#     ax.legend(
#         [Patch(facecolor=regular_color), Patch(facecolor=closed_color)],
#         ["Regular", "Closed"],
#         loc="lower right",
#     )

# # Plot the each year sub-plot
# grid.map_dataarray(plot_mangrove, x="x", y="y", add_colorbar=False)

# # Update sub-plot titles
# for i, name in np.ndenumerate(grid.name_dicts):
#     grid.axes[i].title.set_text(str(name["year"]));

In [ ]:
# # Convert pixel count to km^2
# m2_per_ha = 10000
# m2_per_pixel = query["resolution"][1] ** 2
# mangrove_area = mangroves.sum(dim=("x", "y")) * m2_per_pixel / m2_per_ha

# # Fix axis and legend text
# mangrove_area.name = r"Area (Ha)"
# mangrove_area = mangrove_area.rename(mangrove_type="Mangrove type")

# # Plot the line graph
# mangrove_area.plot(
#     hue="Mangrove type",
#     x="year",
#     xticks=mangrove_area.year,
#     size=6,
#     linestyle="--",
#     marker="o",
# )
# plt.title("Mangrove classification over time");

In [ ]:
total_mangroves = (mangroves.loc["total"] == 1).astype(int)

# Calculate the change in mangrove extent
old = total_mangroves.isel(year=0)
new = total_mangroves.isel(year=-1)
change = new - old

# reclassify into growth, loss and stable
growth = xr.where(change == 1, 1, np.nan)
loss = xr.where(change == -1, -1, np.nan)
stable = old.where(~change)
stable = xr.where(stable == 1, 1, np.nan)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

ds_summaries.isel(year=0).plot.imshow(
    ax=ax, cmap="Greys", vmin=-1, vmax=1, add_colorbar=False, add_labels=False
)
stable.plot(
    ax=ax, cmap=ListedColormap(["palegoldenrod"]), add_colorbar=False, add_labels=False
)
growth.squeeze().plot.imshow(
    ax=ax, cmap=ListedColormap(["lime"]), add_colorbar=False, add_labels=False
)
loss.plot.imshow(
    ax=ax, cmap=ListedColormap(["fuchsia"]), add_colorbar=False, add_labels=False
)
ax.legend(
    [
        Patch(facecolor='lime'),
        Patch(facecolor='fuchsia'),
        Patch(facecolor="palegoldenrod"),
    ],
    ["New mangroves", "Loss of mangroves", "Stable mangroves"],
    loc="lower right",
)
plt.title('Change in mangrove extent between {} and {}'.format(ds_summaries.year.values[0], ds_summaries.year.values[-1]));

In [ ]:
loss_count = loss.sum().values * m2_per_pixel / m2_per_ha
gain_count = growth.sum().values * m2_per_pixel / m2_per_ha
stable_count = stable.sum().values* m2_per_pixel / m2_per_ha

counts = {
          'Loss of Mangroves': loss_count,
          'New Mangroves': gain_count,
          'No Change': stable_count,
          'Delta Change': gain_count + loss_count
          }

df = pd.DataFrame(counts, index=['Area (Ha)'])
df.plot.barh(color=['tab:red','tab:green','tab:orange','tab:blue'])
plt.title('Change in mangrove extent between {} and {}'.format(ds_summaries.year.values[0], ds_summaries.year.values[-1]));

In [ ]:
dataset = ds

rgb(ds, bands=['B11_20m', 'B08_10m', 'B02_10m'], index=[0, 1])

In [ ]:
# #calculate NDVI, rename nir2 to trick calculate_indices
# ds_filtered = ds_filtered.rename({'B0':'nir'})
# ds_filtered = calculate_indices(ds_filtered, index='NDVI', satellite_mission='s2')

In [ ]:
ds_summaries = ds_filtered.NDVI.groupby("time.year").median().compute()

# Plot the output summary images
ds_summaries.plot(col="year", cmap="YlGn", col_wrap=len(ds_summaries.year.values), vmin=0, vmax=1.0);

In [ ]:
# import os

# # Set the path to the configuration file
# config_path = '/path/to/config_file.yaml'

# # Set the TIDE_CONFIG_LOCATION environment variable
# os.environ['TIDE_CONFIG_LOCATION'] = config_path

In [ ]:
# import numpy as np
# from astral.sun import sun
# from astral import LocationInfo
# import datetime

# # define location
# # latitude = -12.345
# # longitude = 123.456

# latitude=0
# longitude=0

# location = LocationInfo('Greenwich', 'India', latitude, longitude)

# # define time range
# start_time = '2022-01-01'
# end_time = '2023-01-10'
# times = np.arange(start_time, end_time, dtype='datetime64[D]')
# # print(times)
# dt = times.astype(datetime.datetime)
# # print(dt[0].year)
# # calculate tides
# tides = []
# for time in dt:
#     # get solar position
#     s = sun(location.observer, date = time)
#     print(s)
#     altitude = s.dawn
#     azimuth = s.azimuth

#     # calculate tide
#     tide = (altitude - 12) / 12  # simple linear equation
#     tides.append(tide)

# # plot tides
# import matplotlib.pyplot as plt
# plt.plot(dt, tides)
# plt.xlabel('Time')
# plt.ylabel('Tide')
# plt.show()
# import numpy as np
# from pytides.constituent import tidal_constituents
# from pytides.tide import Tide

# # Define the location of interest
# lat = -12.3456  # latitude in decimal degrees
# lon = 123.4567  # longitude in decimal degrees

# # Define the time range of interest
# start_time = '2022-01-01'
# end_time = '2022-01-31'
# times = np.arange(np.datetime64(start_time), np.datetime64(end_time), np.timedelta64(1, 'h'))

# # Create a Tide object
# tide = Tide.decompose(constituent=tidal_constituents,
#                       lat=lat,
#                       lon=lon,
#                       times=times.astype('datetime64[ns]'))

# # Get the tidal heights at each time step
# tide_heights = tide.predict(times.astype('datetime64[ns]'))


In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import pytz
import astral
from astral.sun import sun

# Define the location of interest
latitude = -40.8
longitude = 151.2

# Define the time range of interest
start_time = '2022-01-01'
end_time = '2022-12-31'

# Load the datacube dataset
# Note: replace this with your own code for loading datacube data
dc_dataset = ds
location = astral.LocationInfo('Greenwich', 'India', latitude, longitude)

# Calculate sunrise and sunset times for each day within the time range
timezone = pytz.timezone('Australia/Sydney')
sun_times = []
for date in pd.date_range(start_time, end_time):
    sun = sun(location.observer, date, latitude, longitude)
    sunrise = timezone.localize(sun['sunrise']).strftime('%Y-%m-%d %H:%M:%S')
    sunset = timezone.localize(sun['sunset']).strftime('%Y-%m-%d %H:%M:%S')
    sun_times.append((date.strftime('%Y-%m-%d'), sunrise, sunset))

sun_times_df = pd.DataFrame(sun_times, columns=['date', 'sunrise', 'sunset'])

# Calculate the time of the lowest astronomical tide (LAT) for each day within the time range
lat_times = []
for _, row in sun_times_df.iterrows():
    sunset_time = datetime.strptime(row['sunset'], '%Y-%m-%d %H:%M:%S')
    lat_time = sunset_time - pd.Timedelta(hours=6)
    lat_times.append(lat_time.strftime('%Y-%m-%d %H:%M:%S'))

lat_times_df = pd.DataFrame({'date': sun_times_df['date'], 'lat_time': lat_times})

# Convert LAT times to the same time zone as the datacube dataset
lat_times_df['lat_time'] = pd.to_datetime(lat_times_df['lat_time']).dt.tz_localize(pytz.utc).dt.tz_convert(timezone)

# Filter the datacube dataset for only times when the tide is at its lowest point
dc_dataset = dc_dataset.sel(time=lat_times_df['lat_time'])

# Apply cloud masking algorithms to the filtered datacube dataset to remove clouds and cloud shadows
# Note: replace this with your own code for cloud masking
masked_dc_dataset = ...

# Perform your analysis on the resulting cloud-free datacube dataset
# Note: replace this with your own analysis code
analysis_result = ...
